In [ ]:
import piplite
await piplite.install("numpy")
await piplite.install("matplotlib")
await piplite.install("pytest")

<br><br><br>

## Orbitty

In [ ]:
from my_cool_package import orbitty

<br><br><br><br><br>

This is a little simulation of gravitational forces.

Look! A comet!

In [ ]:
system = orbitty.System(
    m=[1000, 1], x=[[0, 0, 0], [0, 10, 0]], p=[[10, 0, 0], [-10, 0, 0]]
)
system.steps(300)
system.plot()

A moon!

In [ ]:
system = orbitty.System(
    m=[1000, 1, 1],
    x=[[0, 0, 0], [0, 10, 0], [0, 11, 0]],
    p=[[0, 0, 0], [-16, 0, 0], [-13, 0, 0]],
)
system.steps(600)
system.plot()

The three-body problem!

In [ ]:
p1 = 0.347111
p2 = 0.532728
system = orbitty.System(
    m=[1, 1, 1],
    x=[[-1, 0, 0], [1, 0, 0], [0, 0, 0]],
    p=[[p1, p2, 0], [p1, p2, 0], [-2 * p1, -2 * p2, 0]],
)
system.G = 1
system.steps(1000, dt=0.01)
system.plot()

A whole lot of particles!

In [ ]:
system = orbitty.System.random(
    num_particles=20,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=10,
)
system.steps(10000, dt=0.1)
system.plot()

<br><br><br><br><br>

## How would you write tests for _this_?

It's hard to test a plot.

Even if you get the plot as an image or a video file and test it against an expected image/video,

* you'd have to compare the _uncompressed_ image/video, since minor differences in codec versions can make the compressed bytes differ when nothing has really changed,
* the test might be run on a computer with different plotting backend, making the plots differ in irrelevant ways (margins, fonts, ...),
* if you ever want to add tests, you'll have to go through a complicated process of making expected images/videos.

It's not worth it!

<br><br><br><br><br>

We have to somehow get raw values:

In [ ]:
import numpy as np

In [ ]:
system = orbitty.System(
    m=[1000, 1], x=[[0, 0, 0], [0, 10, 0]], p=[[10, 0, 0], [-10, 0, 0]]
)
system.steps(1000)

In [ ]:
system.t_history

In [ ]:
system.x_history

In [ ]:
system.p_history

<br><br><br><br><br>

These arrays are too big to write verbatim in the source code (8950 lines after formatting), but not too large to check into git (71 kB).

Save them in some stable format (JSON, Pickle, HDF5, ROOT, ...) and check them into the repo.

I've already done this, so let's just check.

In [ ]:
import pickle

In [ ]:
with open("tests/samples/orbitty-samples.pkl", "rb") as file:
    t_history, x_history, p_history = pickle.load(file)

In [ ]:
np.testing.assert_allclose(system.t_history, t_history)

In [ ]:
np.testing.assert_allclose(system.x_history, x_history)

In [ ]:
np.testing.assert_allclose(system.p_history, p_history)

<br><br><br><br><br>

### Property-based testing

Physics question: what are the invariants of motion for this physical system?

In [ ]:
import pytest

#### One

In [ ]:
def total_momentum(system):
    return np.sum(system.p, axis=0).tolist()

#### Two

In [ ]:
def total_energy(system):
    # KE = 1/2 m |v|^2 and p = mv, so KE = 1/2 |p|^2 / m
    kinetic = np.sum(0.5 * np.sum(system.p**2, axis=1) / system.m)

    # gravitational force -> potential integration in 3D
    assert system.num_dimensions == 3
    # indexes to pick out (particle 1, particle 2) pairs, for all pairs
    p1, p2 = np.triu_indices(len(system.x), k=1)
    # pairwise (pw) displacements between all particle pairs
    pw_displacement = system.x[p2] - system.x[p1]
    # pairwise displacement is a sum in quadrature over all dimensions
    pw_distance = np.sqrt(np.sum(pw_displacement**2, axis=-1))
    # PE = -G m1 m2 / distance (for each pair)
    pw_potential = -system.G * system.m[p1] * system.m[p2] / pw_distance
    # sum over pairs to get the potential for each particle
    particle_potential = np.zeros_like(system.m)
    np.add.at(particle_potential, p1, pw_potential)
    np.add.at(particle_potential, p2, pw_potential)
    # avoid double-counting (particle 1, particle 2) and (particle 2, particle 1)
    potential = 0.5 * np.sum(particle_potential)

    return potential + kinetic

#### Three

In [ ]:
def center_of_mass(system):
    return np.sum(
        system.m[:, np.newaxis] * system.x / np.sum(system.m), axis=0
    ).tolist()

will move linearly.

If the initial momentum is zero, then this is a constant.

#### More?

Maybe Kepler's laws of motion, but those are equivalent to the above.

Liouville's theorem! The volume covered by the particles in $\vec{x}$-$\vec{p}$ space should be constant. (I haven't tested this.)

<br><br><br><br><br>

### Applying them

#### Center of mass on the comet

In [ ]:
system = orbitty.System(
    m=[1000, 1], x=[[0, 0, 0], [0, 10, 0]], p=[[10, 0, 0], [-10, 0, 0]]
)

initial = center_of_mass(system)
for i in range(1000):
    system.step()
    assert center_of_mass(system) == initial, f"{i}\n{center_of_mass(system)}\n{initial}"

What are we missing?

<br><br><br><br><br>

#### Total momentum on the moon

In [ ]:
system = orbitty.System(
    m=[1000, 1, 1],
    x=[[0, 0, 0], [0, 10, 0], [0, 11, 0]],
    p=[[0, 0, 0], [-16, 0, 0], [-13, 0, 0]],
)

initial = total_momentum(system)
for i in range(1000):
    system.step()
    assert total_momentum(system) == pytest.approx(
        initial
    ), f"{i}\n{total_momentum(system)}\n{initial}"

<br><br><br>

#### Total energy on the three-body problem

In [ ]:
p1 = 0.347111
p2 = 0.532728
system = orbitty.System(
    m=[1, 1, 1],
    x=[[-1, 0, 0], [1, 0, 0], [0, 0, 0]],
    p=[[p1, p2, 0], [p1, p2, 0], [-2*p1, -2*p2, 0]]
)
system.G = 1

initial = total_energy(system)
for i in range(1000):
    system.step()
    assert total_energy(system) == pytest.approx(initial), f"{i}\n{total_energy(system)}\n{initial}"

Hmmm. That's really close.

Try loosening the tolerance on [pytest.approx](https://docs.pytest.org/en/latest/reference/reference.html#pytest-approx).

<br><br><br><br><br>

Is that good enough?

Remember that automated tests are _for your benefit_. The question is: what would convince _you_ that there's truly something wrong and not some round-off error?

What about reducing the step size in the numerical simulation, instead of widening the tolerance for error?

<br><br><br><br><br>

### Property tests with random inputs

In [ ]:
rng = np.random.default_rng(seed=12345)

system = orbitty.System.random(
    num_particles=20,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=10,
    rng=rng,
)

initial = total_momentum(system)
for i in range(10000):
    system.step(dt=0.1)
    assert total_momentum(system) == pytest.approx(
        initial
    ), f"{i}\n{total_momentum(system)}\n{initial}"

Out of curiosity, what did that simulation look like?

In [ ]:
system.plot()

The fact that the total momentum remained constant, while the particles all weave around each other in 3D, is an impressive demonstration that it's correct.

<br><br><br><br><br>

## Let's see those tests!

* [tests/test_0003_orbitty.py](tests/test_0003_orbitty.py)

In [ ]:
pytest.main(["-v", "tests/test_0003_orbitty.py"])

<br><br><br><br><br>

Did that take too long?

Again, it's up to you. The tests are there for _your_ benefit!

* If they take so long that you never want to run them, then they're not useful.
* If they don't test all the cases or test too superficially, then they're not useful.

<br><br><br><br><br>

Now let's see what these tests look like in GitHub Actions.

* [https://github.com/jpivarski-talks/my-cool-package/tree/main/tests](https://github.com/jpivarski-talks/my-cool-package/tree/main/tests)
* [https://github.com/jpivarski-talks/my-cool-package/tree/main/.github/workflows](https://github.com/jpivarski-talks/my-cool-package/tree/main/.github/workflows)
* [https://github.com/jpivarski-talks/my-cool-package/actions](https://github.com/jpivarski-talks/my-cool-package/actions)